<a href="https://colab.research.google.com/github/alecseiterr/safe_city/blob/main/common_python_scripts/Grounding_Dino_Labeling_for_YOLO8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Модуль разбора видео на кадры

В этом модуле автоматически разбираются видеофайлы на отдельные кадры и складываются в папку на примонтированном Google Drive. Эти изображения далее идут в следующий модуль детекции сущностей для формирования датасета в формате Yolo8.

__Параметры:__

step_ms = 250 # Интервал извлечения кадров в мсек

source = 'local' или 'YouTube' # Читаем файл из примонтированной папки Google Drive или из YouTube

capture_path = '/content/drive/MyDrive/UII/Capture/' # Куда сохраняются нарезанные изображения кадров

video_path = '/content/drive/MyDrive/UII/Video/' # Где хранятся разбираемые видео (разберутся все видео, какие лежат в папке, внимательно следите за ее содержимым)

video_url = "https://www.youtube.com/watch?v=5wwH7Hll0xE&pp=ygUOWW9nYSB0aW1lbGFwc2U%3D"  # Пример ссылки на ролик в Youtube, который хотим разобрать (по окончанию работы видеофайл .mp4 будет тоже лежать в video_path)


In [ ]:
# Функция очистки папки от предыдущих результатов
def cleaner_folder(path):
    for file_name in os.listdir(path):
        file_path = os.path.join(path, file_name)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Не удалось удалить %s. Причина: %s' % (file_path, e))

In [ ]:
# Функция удаления директории целиком
def delete_directory(path: str):
  if os.path.exists(path):
    try:
      shutil.rmtree(path)
      # print(f'[INFO] Directory was successfully deleted: `{path}`')
    except OSError as e:
      print(f'[ERROR] Error deleting a directory: `{path}`: {e.strerror}')
      return

In [ ]:
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.9 MB/s eta 0:00:00


In [ ]:
import cv2
import os
import shutil
from send2trash import send2trash
from google.colab import drive
drive.mount('/content/drive')
from pytube import YouTube
from tqdm import tqdm
!rm -r sample_data

Mounted at /content/drive


In [ ]:
step_ms = 500 # Интервал извлечения кадров в мсек

# Параметр source определяет откуда будем парсить: из YouTube или сохраненное на Google Disk

#source = 'local' # Читаем файл из Google Drive, иначе из YouTube
source = 'YouTube'

capture_path = '/content/drive/MyDrive/UII/Capture/'
video_path = '/content/drive/MyDrive/UII/Video/'

#video_url = "https://www.youtube.com/watch?v=5wwH7Hll0xE&pp=ygUOWW9nYSB0aW1lbGFwc2U%3D"  # YouTube Yoga Timelapse
#video_url = "https://www.youtube.com/watch?v=0pKQ8E5cFIE"  # Подскальзывания
#video_url = "https://www.youtube.com/watch?v=5xKwYVlpmpY" # Падения
#video_url = "https://www.youtube.com/watch?v=GZcghScPe6Y" # Подборка разных падений
#video_url = "https://www.youtube.com/watch?v=o-N3ImykVuI" # Падения разные
#video_url = "https://www.youtube.com/watch?v=S_aderfA1EM" # Курение кино
video_url = "https://www.youtube.com/watch?v=iks6yE0dBkM" # Курение Шелби

# Функция очистки папки от предыдущих результатов
def cleaner_folder(path):
    for file_name in os.listdir(path):
        file_path = os.path.join(path, file_name)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Не удалось удалить %s. Причина: %s' % (file_path, e))

# Проверка существования пути для видео, если нет, то создаем
if not os.path.exists(video_path):
    os.makedirs(video_path)

# Проверка существования пути для кадров, если нет, то создаем
if not os.path.exists(capture_path):
    os.makedirs(capture_path)

# очистим папку Capture от предыдущих результатов
#cleaner_folder(capture_path)

# Функция для скачивания видео с YouTube
def download_youtube_video(url, path):
    yt = YouTube(url)
    # Выбор потока с наивысшим разрешением
    video_stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    video_stream.download(path)
    return os.path.join(path, video_stream.default_filename)

# Функция выделения кадров из видео каждые step_ms
def extract_frames(video_file, step_ms):
    vidcap = cv2.VideoCapture(video_file)
    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = vidcap.get(cv2.CAP_PROP_FPS)
    duration = total_frames / frame_rate
    total_steps = int(duration * 1000 / step_ms)

    print(f"Начало обработки видео: {video_file}")

    count = 0
    with tqdm(total=total_steps, desc="Обработка", unit="кадр") as pbar:
        while count < total_steps:
            vidcap.set(cv2.CAP_PROP_POS_MSEC, (count * step_ms))
            success, image = vidcap.read()
            if success:
                frame_filename = f"{os.path.splitext(os.path.basename(video_file))[0]}_frame{count}.png"
                cv2.imwrite(os.path.join(capture_path, frame_filename), image)
            pbar.update(1)
            count += 1

    print(f"\nОбработка видео завершена. Результаты лежат в: {capture_path}")

if source == 'local':
    # Чтение и обработка каждого видеофайла в папке
    for filename in os.listdir(video_path):
        if filename.endswith(".mp4"): # Проверка, что файл является видео
            extract_frames(os.path.join(video_path, filename), step_ms)
else:
    downloaded_video_path = download_youtube_video(video_url, video_path)
    extract_frames(downloaded_video_path, step_ms)


Начало обработки видео: /content/drive/MyDrive/UII/Video/Если курить то только как Томас Шелби.mp4


Обработка: 100%|██████████| 270/270 [01:01<00:00,  4.42кадр/s]


Обработка видео завершена. Результаты лежат в: /content/drive/MyDrive/UII/Capture/


## Модуль детекции объектов на картинках по текстовому описанию. Используется Grounding Dino

В этом модуле производится автоматическая детекция сущностей по текстовому промту целиком во всей папке с сырыми изображениями. Результат сохраняется в виде размеченного датасета по структуре Yolo8.

__Параметры:__

BOX_TRESHOLD = 0.43 # Порог вероятности обнаружения искомой сущности

TEXT_TRESHOLD = 0.45 # Тоже параметрический порого grounding dino. Влияния на результат детекции не обнаружил, можно не менять.

TEXT_PROMPT = 'fall' # Текстовое название сущности, которую хотим детектить на изображениях.

capture_path = '/content/drive/MyDrive/UII/Capture/' # Откуда берем картинки на анализ (результат работы Молуля 1 или можно залить по этому пути любые свои картинки)

annotated_path = '/content/drive/MyDrive/UII/Capture/Annotated/' # Куда складываются изображения с обнаруженными сущностями. На картинках наложены прямоугольноки bbox/. На Шаге 2 именно в этой папке вручную удаляем лишние изображения.

yolo_path = '/content/drive/MyDrive/UII/Capture/Yolo8/' # Куда сохранится итоговый датасет в формате Yolo8.

### Шаг подготовки окружения, модели, весов, путей, рабочих функций.

In [ ]:
# Установка Grounding Dino

!pip install wget
import os
HOME = os.getcwd()
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!pip install -e .
# загрузка весов GroundingDINO
%cd {HOME}
os.makedirs(os.path.join(HOME, 'weights'), exist_ok=True)
%cd {HOME}/weights
url = 'https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth'
import wget
wget.download(url, os.getcwd())
# Загрузка модели GroundingDINO
%cd {HOME}/GroundingDINO

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=5cac612c2115e9e30e019f83922f27d2c7180ada81d2b440f77933211590bc2a
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
/content
Cloning into 'GroundingDINO'...
remote: Enumerating objects: 421, done.
remote: Counting objects: 100% (190/190), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 421 (delta 145), reused 128 (delta 128), pack-reused 231
Receiving objects: 100% (421/421), 12.85 MiB | 19.08 MiB/s, done.
Resolving deltas: 100% (216/216), done.
/content/GroundingDINO
Obtaining file:///content/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 13.2 M

In [ ]:
import torch
import numpy as np
import random
import os
import shutil
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from groundingdino.util.inference import load_model, load_image, predict, annotate
import supervision as sv
from google.colab import drive
drive.mount('/content/drive')
from google.colab.patches import cv2_imshow
from PIL import Image, ImageDraw
%matplotlib inline
import math
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
!rm -r sample_data

Mounted at /content/drive
rm: cannot remove 'sample_data': No such file or directory


In [ ]:
# глобальные параметры и переменные

CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
TEXT_TRESHOLD = 0.35

# Задаем рабочие пути
capture_path = '/content/drive/MyDrive/UII/Capture/'
annotated_path = '/content/drive/MyDrive/UII/Capture/Annotated/'
yolo_path = '/content/drive/MyDrive/UII/Capture/Yolo8/'

# Проверка и создание путей для аннотированных изображений
if not os.path.exists(annotated_path):
    os.makedirs(annotated_path)
if not os.path.exists(yolo_path):
    os.makedirs(yolo_path)

/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True
/content/weights/groundingdino_swint_ogc.pth ; exist: True


Функции для распознавания через grounding dino

In [ ]:
# Функция преобразования координат меток для визуализации
def transform_bbox_coords(bbox):
    x_cent, y_cent, width, height = bbox
    x_min = int(x_cent - width/2)
    y_min = int(y_cent - height/2)
    x_max = int(x_cent + width/2)
    y_max = int(y_cent + height/2)
    return [x_min, y_min, x_max, y_max]

# Функция детекции сущностей на изображении и формирования данных в формате yolo
def gdino_inference(image_path, text_prompts, bbox_lines_whidth=2, different_bbox_colors_on_image=False, return_image_only=False):
    image_source, image = load_image(image_path)
    bbox_data_for_txt = []  # Для сохранения данных bbox для YOLO
    imagePIL = Image.fromarray(image_source)  # Инициализация imagePIL
    draw = ImageDraw.Draw(imagePIL) if not return_image_only else None
    detected_objects = False  # Флаг обнаружения объектов

    for id_class, (text_prompt, box_threshold) in text_prompts.items():
        boxes, logits, phrases = predict(model=model, image=image, caption=text_prompt, box_threshold=box_threshold, text_threshold=TEXT_TRESHOLD, remove_combined = True)
        if len(boxes) > 0:
            detected_objects = True  # Объекты обнаружены
            width, height = imagePIL.size
            bboxes = (boxes * torch.Tensor(imagePIL.size).tile((boxes.size()[0], int(boxes.size()[1]/2)))).to(dtype=torch.int16).tolist()
            color = tuple(np.random.randint((255, 255, 255)))

            for bbox in bboxes:
                transformed_bbox = transform_bbox_coords(bbox)
                if not return_image_only:
                    if different_bbox_colors_on_image:
                        color = tuple(np.random.randint((255, 255, 255)))
                    draw.rectangle(transformed_bbox, outline=color, width=bbox_lines_whidth)

                x_cent, y_cent, w, h = bbox
                x_cent, y_cent, w, h = x_cent / width, y_cent / height, w / width, h / height
                bbox_data_for_txt.append(f'{id_class} {x_cent} {y_cent} {w} {h}')

    return imagePIL, detected_objects, bbox_data_for_txt


Вспомогательные функции для манипуляций с файлами и структурой yolo

In [ ]:
# Функция сохранения данных bbox
def save_bbox_data(filename, bbox_data, save_path):
    txt_filename = filename.replace('.png', '.txt')
    with open(os.path.join(save_path, txt_filename), 'w') as file:
        for line in bbox_data:
            file.write(line + '\n')

# Функция для копирования и создания .txt файлов в yolo_path
def prepare_yolo_dataset():
    for filename in os.listdir(capture_path):
        if filename.endswith(".png") and os.path.exists(os.path.join(annotated_path, filename)):
            # Копирование изображения
            shutil.copy(os.path.join(capture_path, filename), os.path.join(yolo_path, 'train/images', filename))
            # Чтение и копирование данных bbox
            txt_filename = filename.replace('.png', '.txt')
            shutil.copy(os.path.join(capture_path, txt_filename), os.path.join(yolo_path, 'train/labels', txt_filename))

# Функция для создания структуры Yolo8: изображения, метки
def create_yolo_directory_structure():
    train_path = os.path.join(yolo_path, 'train')
    images_path = os.path.join(train_path, 'images')
    labels_path = os.path.join(train_path, 'labels')

    os.makedirs(images_path, exist_ok=True)
    os.makedirs(labels_path, exist_ok=True)

# Функция очистки папки от предыдущих результатов
def cleaner_folder(path):
    for file_name in os.listdir(path):
        file_path = os.path.join(path, file_name)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Не удалось удалить %s. Причина: %s' % (file_path, e))

In [ ]:
# Создание экземпляра модели
model = load_model(CONFIG_PATH, WEIGHTS_PATH)

final text_encoder_type: bert-base-uncased


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

### Шаг 1: Обработка исходных изображений и аннотация с наложенными прямоугольниками bbox. Плюс сохранение параметров bbox.

На этом шаге обрабатываются изображения с наложением на них bbox. Сохраняются аннотированные изображения в annotated_path для визуальной оценки на Шаге 2. Одновременно координаты bbox сохраняются в памяти для использования на Шаге 3.

In [ ]:
TEXT_PROMPTS = {
    5: ['cigarette', 0.34],
 #   1: ['smoke.fume.steam', 0.39],
 #   2: ['person', 0.68],
 #   3: ['fire', 0.35],
 #   4: ['helmet', 0.55],
 #   5: ['vest', 0.52]
}

cleaner_folder(annotated_path)

# Словарь для хранения данных bbox всех обработанных изображений
bbox_data_dict = {}

# Основной рабочий цикл для аннотации изображений
for filename in tqdm(os.listdir(capture_path), desc="Обработка кадров", unit="кадр", leave=True):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')):
        image_path = os.path.join(capture_path, filename)

        result_image, object_detected, bbox_data = gdino_inference(image_path, TEXT_PROMPTS, return_image_only=False)

        if object_detected:
            annotated_image_path = os.path.join(annotated_path, filename)
            # Сохранение аннотированного изображения
            result_image.save(annotated_image_path)

            # Сохранение данных bbox в словарь
            bbox_data_dict[filename] = bbox_data

Обработка кадров: 100%|██████████| 748/748 [16:59<00:00,  1.36s/кадр]


### Шаг 2: Ручная очистка аннотированных изображений

Вы вручную удаляете неподходящие изображения из папки Annotated.

### Шаг 3: Создание структуры YOLO и копирование подходящих изображений и параметров bbox.

На этом шаге в /Yolo8/train/images копируются только те изображения из capture_path, которые совпадают по названию с изображениями в annotated_path, и в /Yolo8/train/labels создаются соответствующие .txt файлы с координатами bbox.

In [ ]:
# Создание структуры папок YOLO
create_yolo_directory_structure()

# Копирование изображений и создание .txt файлов в папки YOLO
for filename in tqdm(os.listdir(annotated_path), desc="Сохранение датасета YOLOv8", unit="кадр"):
    if filename.endswith(".png"):
        original_image_path = os.path.join(capture_path, filename)

        if os.path.exists(original_image_path):
            # Копирование изображения
            shutil.copy(original_image_path, os.path.join(yolo_path, 'train/images', filename))

            # Создание и сохранение данных bbox в .txt файл
            if filename in bbox_data_dict:
                save_bbox_data(filename, bbox_data_dict[filename], os.path.join(yolo_path, 'train/labels'))

# Датасет в формате yolo8 создан, поэтому промежуточная папка annotated_path нам больше не нужна. Удаляем ее содержимое.
#cleaner_folder(annotated_path)

Сохранение датасета YOLOv8: 100%|██████████| 715/715 [00:00<00:00, 447067.29кадр/s]
